In [1]:
# Instalando e atualizando as bibliotecas necessárias
!pip install pyDOE
!pip install -U seaborn

Requirement already up-to-date: seaborn in c:\users\wande\anaconda3\lib\site-packages (0.11.2)


In [2]:
pip install -i https://test.pypi.org/simple/ FINITO-FEM-TOOLBOX

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
from RASD_TOOLBOX import *
from FINITO_FEM_TOOLBOX import MEF1D
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
def MODULO_ELASTICIDADE_CONCRETO(AGREGADO, F_CK, F_CKJ):
    """
    Esta função calcula os módulos de elasticidade do concreto.  

    Entrada:
    AGREGADO    | Tipo de agragado usado no traço do cimento       |        | string    
                |   'BAS' - Agregado de Basalto                    |        | 
                |   'GRA' - Agregado de Granito                    |        |              
                |   'CAL' - Agregado de Calcário                   |        |
                |   'ARE' - Agregado de Arenito                    |        | 
    F_CK        | Resistência característica à compressão          | kN/m²  | float   
    F_CKJ       | Resistência característica à compressão idade J  | kN/m²  | float
    
    Saída:
    E_CIJ       | Módulo de elasticidade tangente                  | kN/m²  | float
    E_CSJ       | Módulo de elasticidade do secante                | kN/m²  | float   
    """
    # Determinação do módulo tangente E_CI idade T
    if AGREGADO == 'BAS':         
        ALFA_E = 1.2
    elif AGREGADO == 'GRA':         
        ALFA_E = 1.0
    elif AGREGADO == 'CAL':       
        ALFA_E = 0.9
    elif AGREGADO == 'ARE':       
        ALFA_E = 0.7
    F_CK /= 1E3
    if F_CK <= 50:        
        E_CI = ALFA_E * 5600 * np.sqrt(F_CK)
    elif F_CK > 50:   
        E_CI = 21.5 * (10 ** 3) * ALFA_E * (F_CK / 10 + 1.25) ** (1 / 3)
    ALFA_I = 0.8 + 0.2 * F_CK / 80
    if ALFA_I > 1:        
        ALFA_I = 1
    # Determinação do módulo secante E_CS idade T
    E_CS = E_CI * ALFA_I
    if F_CK <= 45 :
        F_CK *= 1E3
        E_CIJ = E_CI * (F_CKJ / F_CK) ** 0.5  
    elif  F_CK > 45 : 
        F_CK *= 1E3
        E_CIJ = E_CI * (F_CKJ / F_CK) ** 0.3  
    E_CSJ = E_CIJ * ALFA_I
    E_CIJ *= 1E3 
    E_CSJ *= 1E3 
    return E_CIJ, E_CSJ

def MOMENTO_RESISTENE_MRD(B_W, D, A_S, F_YK, F_CK, GAMMA_M_ACO, GAMMA_M_CONCRETO):
    """
    Esta função determina o momento resistente M_RD de uma viga de concreto armado.

    Entrada:
    B_W                | Largura da viga                               | m     | float  
    D                  | Altura útil da seção                          | m     | float
    A_S                | Área de aço necessária na seção               | m²    | float
    F_YK               | Tensão de escoamento do aço                   | kN/m² | float
    GAMMA_M_ACO        | Coeficiente parcial de segurança do aço       |       | float
    GAMMA_M_CONCRETO   | Coeficiente parcial de segurança do concreto  |       | float

    Saída:
    M_RD               | Momento resistente da peça                    | kN.m  | Float 
    """
    # Definição dos parâmetros que dependem do F_CK
    F_CK /= 1E3
    if F_CK >  50:
        LAMBDA = 0.80 - ((F_CK - 50) / 400)
        ALPHA_C = (1.00 - ((F_CK - 50) / 200)) * 0.85
    else:
        LAMBDA = 0.80
        ALPHA_C = 0.85
    F_CK *= 1E3    
    F_CD = F_CK / GAMMA_M_CONCRETO
    F_YD = F_YK / GAMMA_M_ACO
    # Linha neutra real e momento resistente
    X_III = (A_S * F_YD) / (ALPHA_C * F_CD * B_W * LAMBDA)
    M_RD = A_S * F_YD * (D - 0.50 * LAMBDA * X_III)
    return M_RD

def CORTANTE_RESISTENE_VRD2(F_CK, B_W, D, GAMMA_M_CONCRETO):
    """
    Esta função verifica o valor da resistência da biela comprimida V_RD2.

    Entrada:
    F_CK               | Resistência característica à compressão         | kN/m² | float
    B_W                | Largura da viga                                 | m     | float  
    D                  | Altura útil da seção                            | m     | float
    GAMMA_M_CONCRETO   | Coeficiente parcial de segurança do concreto    |       | float

    Saída:
    V_RD2              | Força resistente da biela comprimida            | kN    | float
    """
    V_RD2 = RESISTENCIA_BIELA_COMPRIMIDA(F_CK, B_W, D, GAMMA_M_CONCRETO)
    return V_RD

def RESISTENCIA_BIELA_COMPRIMIDA(F_CK, B_W, D, GAMMA_M_CONCRETO):
    """
    Esta função verifica o valor da resistência da biela comprimida V_RD2.

    Entrada:
    F_CK               | Resistência característica à compressão         | kN/m² | float
    B_W                | Largura da viga                                 | m     | float  
    D                  | Altura útil da seção                            | m     | float
    GAMMA_M_CONCRETO   | Coeficiente parcial de segurança do concreto    |       | float
    
    Saída:
    V_RD2              | Resitência da biela comprimida                  | kN    | float 
    """
    # Força resistente da biela de compressão
    F_CK /= 1E3 
    ALFA_V2 = (1 - (F_CK / 250))
    F_CK *= 1E3 
    F_CD = F_CK / GAMMA_M_CONCRETO
    V_RD2 = 0.27 * ALFA_V2 * F_CD * B_W * D
    return V_RD2

def FLECHA_LIMITE_DELTA_RD(L):
    """
    Esta função verifica o valor da flecha limite para uma viga.

    Entrada:
    L         | Vão da peça analisada           | m     | float
    
    Saída:
    DELTA_RD  | Flecha resistente               | m     | float 
    """
    # Flecha Limite
    DELTA_RD = (L / 250)
    return DELTA_RD

def VERIFICA_PILAR(TIPO_PILAR, A_C, RHO, E_S, F_CK, GAMMA_M_CONCRETO):
    """
    Esta função verifica o valor da carga normal máxima em um pilar de concreto armado.
    
    Entrada:
    TIPO_PILAR        | Tipo de pilar da estrutura                    |       | string
                      |    "INTERMEDIARIO"                            |       | 
                      |    "EXTREMIDADE"                              |       | 
                      |    "CANTO"                                    |       | 
    A_C               | Área da seção de concreto do pilar            | m²    | float
    RHO               | Taxa mecânica de armadura                     | %     | float
    E_S               | Módulo de elasticidade do aço                 | kN/m² | float
    F_CK              | Resistência característica à compressão       | kN/m² | float
    GAMMA_M_CONCRETO  | Coeficiente parcial de segurança do concreto  |       | float
    
    Saída:
    N_RD              | Compressão resistente do pilar                | m     | float 
    """
    # Definição do coeficiente de conversão de flexo-compressão para compressão
    if TIPO_PILAR == 'CANTO':
        ALPHA = 2.5
    elif TIPO_PILAR == 'EXTREMIDADE':
        ALPHA = 2.2
    elif TIPO_PILAR == 'INTERMEDIARIO':
        ALPHA = 1.8
    # Tensão de escoamento do aço a 0.20% de deformação de compressão
    SGIMA_SE = 0.2 / 100 * E_S
    # Carga máxima resistente
    F_CD = F_CK / GAMMA_M_CONCRETO
    RHO /= 100
    N_RD = (A_C * (0.85 * F_CD + RHO * SGIMA_SE)) / ALPHA
    return N_RD

def MOMENTO_FISSURACAO(GEOMETRIC_FACTOR, F_CK, H, X_I, I_I):
    if GEOMETRIC_FACTOR == 0:
        ALPHA = 1.2
    elif GEOMETRIC_FACTOR == 1:
        ALPHA = 1.3
    elif GEOMETRIC_FACTOR == 2:
        ALPHA = 1.5
    F_CK /= 1E3
    if F_CK <= 50:
          F_CTM = 0.3 * F_CK ** (2/3)
    elif F_CK > 50:
          F_CTM = 2.12 * np.log(1 + 0.11 * F_CK)
    F_CT = F_CTM * 1E3
    Y_T = H - X_I
    M_R = ALPHA * F_CT * (I_I / Y_T)
    return M_R

def GEOMETRIC_PROPERTIES_STATE_I(H, B_F, B_W, H_F, A_SB, ALPHA_MOD, D):
    A_C = (B_F - B_W) * H_F + B_W * H + A_SB * (ALPHA_MOD - 1)
    X_I = ((B_F - B_W) * ((H_F ** 2) / 2) + B_W * ((H ** 2 ) / 2) + A_SB * (ALPHA_MOD - 1) * D) / A_C
    I_I = ((B_F - B_W) * H_F ** 3) / 12 + (B_W * H ** 3) / 12 + (B_F - B_W) * H_F * (X_I - H_F / 2) ** 2 + B_W * H * (X_I - H / 2) ** 2 + A_SB * (ALPHA_MOD - 1) * (X_I - D) ** 2
    return A_C, X_I, I_I

def GEOMETRIC_PROPERTIES_STATE_II(H, B_F, B_W, H_F, A_SB, A_ST, ALPHA_MOD, D, D_L):
    if B_F <= B_W:
        A_1 = B_W / 2
        A_2 = H_F * (B_F - B_W) + (ALPHA_MOD - 1) * A_ST + ALPHA_MOD * A_SB
        A_3 = - D_L * (ALPHA_MOD - 1) * A_ST - D * ALPHA_MOD * A_SB - (H_F ** 2) / 2 * (B_F - B_W)
        X_II = (-A_2 + np.sqrt(A_2 ** 2 - 4 * A_1 * A_3)) / (2 * A_1)
    elif B_F > B_W:
        A_1 = B_F / 2
        A_2 = H_F * (0) + (ALPHA_MOD - 1) * A_ST + ALPHA_MOD * A_SB
        A_3 = -D_L*(ALPHA_MOD - 1) * A_ST - D * ALPHA_MOD * A_SB - (H_F ** 2) / 2 * (0)
        X_II = (- A_2 + (A_2 ** 2 - 4 * A_1 * A_3) ** 0.50) / (2 * A_1)
        if X_II > H_F:
            A_1 = B_W / 2
            A_2 = H_F * (B_F - B_W) + (ALPHA_MOD - 1) * A_ST + ALPHA_MOD * A_SB
            A_3 = - D_L * (ALPHA_MOD - 1) * A_ST - D * ALPHA_MOD * A_SB - (H_F ** 2) / 2 * (B_F - B_W)
            X_II = (- A_2 + np.sqrt(A_2 ** 2 - 4 * A_1 * A_3)) / (2 * A_1)    
    if X_II <= H_F:
        I_II = (B_F * X_II ** 3) / 3 + ALPHA_MOD * A_SB * (X_II - D) ** 2 + (ALPHA_MOD - 1) * A_ST * (X_II - D_L) ** 2
    else:
        I_II = ((B_F - B_W) * H_F ** 3) / 12 + (B_W * X_II **3 ) / 3 + (B_F - B_W) * (X_II - H_F / 2) ** 2 + ALPHA_MOD * A_SB * (X_II - D) ** 2 + (ALPHA_MOD - 1) * A_ST * (X_II - D_L) ** 2
    return X_II, I_II

def BRANSON_INERTIA(M_R, M_D,I_I, I_II):
    M_RMD = (M_R / M_D) ** 3
    I_BRANSON = M_RMD * I_I + (1 - M_RMD) * I_II
    return I_BRANSON

In [5]:
# https://www.tqs.com.br/v16/destaques/modelo-vi-projeto-estrutural.html
def ANALISE_MATRICIAL_ELU(DADOS):
    F_I = DADOS['CARGA_NODAL']
    F_II = 2 * DADOS['CARGA_NODAL']
    E_CV = DADOS['E_C'] * 0.40
    E_CP = DADOS['E_C'] * 0.80
    A_V = DADOS['A_V']
    A_P = DADOS['A_P']
    I_V = DADOS['I_V']
    I_P = DADOS['I_P']  
    FRAME_00 = {"TYPE_ELEMENT": 0,
                "TYPE_SOLUTION": 0,
                "N_NODES": 37,
                "N_MATERIALS": 2,
                "N_SECTIONS": 2,
                "N_ELEMENTS": 40,
                "N_DOFPRESCRIPTIONS": 6,
                "N_DOFLOADED": 35,
                "N_DOFSPRINGS": 0,
                "COORDINATES": 
                np.array([
                [0.00,0.00],
                [4.50,0.00],
                [0.00,3.00],
                [0.75,3.00],
                [1.50,3.00],
                [2.25,3.00],
                [3.00,3.00],
                [3.75,3.00],
                [4.50,3.00],
                [0.00,6.00],
                [0.75,6.00],
                [1.50,6.00],
                [2.25,6.00],
                [3.00,6.00],
                [3.75,6.00],
                [4.50,6.00],
                [0.00,9.00],
                [0.75,9.00],
                [1.50,9.00],
                [2.25,9.00],
                [3.00,9.00],
                [3.75,9.00],
                [4.50,9.00],
                [0.00,12.00],
                [0.75,12.00],
                [1.50,12.00],
                [2.25,12.00],
                [3.00,12.00],
                [3.75,12.00],
                [4.50,12.00],
                [0.00,15.00],
                [0.75,15.00],
                [1.50,15.00],
                [2.25,15.00],
                [3.00,15.00],
                [3.75,15.00],
                [4.50,15.00]]),

                "ELEMENTS": #NODE 1,NODE 2,MATERIAL ID,SECTION ID,HINGE ID NODE 1,HINGE ID NODE 2
                np.array([
                [0,2,1,1,0,0],#0
                [2,3,0,0,0,0],#1
                [3,4,0,0,0,0],#2
                [4,5,0,0,0,0],#3
                [5,6,0,0,0,0],#4
                [6,7,0,0,0,0],#5
                [7,8,0,0,0,0],#6
                [8,1,1,1,0,0],#7
                [2,9,1,1,0,0],#8
                [9,10,0,0,0,0],#9
                [10,11,0,0,0,0],#10
                [11,12,0,0,0,0],#11
                [12,13,0,0,0,0],#12
                [13,14,0,0,0,0],#13
                [14,15,0,0,0,0],#14
                [15,8,1,1,0,0],#15
                [9,16,1,1,0,0],#16
                [16,17,0,0,0,0],#17
                [17,18,0,0,0,0],#18
                [18,19,0,0,0,0],#19
                [19,20,0,0,0,0],#20
                [20,21,0,0,0,0],#21
                [21,22,0,0,0,0],#22
                [22,15,1,1,0,0],#23
                [16,23,1,1,0,0],#24
                [23,24,0,0,0,0],#25
                [24,25,0,0,0,0],#26
                [25,26,0,0,0,0],#27
                [26,27,0,0,0,0],#28
                [27,28,0,0,0,0],#29
                [28,29,0,0,0,0],#30
                [29,22,1,1,0,0],#31
                [23,30,1,1,0,0],#32
                [30,31,0,0,0,0],#33
                [31,32,0,0,0,0],#34
                [32,33,0,0,0,0],#35
                [33,34,0,0,0,0],#36
                [34,35,0,0,0,0],#37
                [35,36,0,0,0,0],#38
                [36,29,1,1,0,0]]),#39

                "MATERIALS": #YOUNG, POISSON,DENSITY, THERMAL COEFFICIENT
                np.array([[E_CV, 1, 1, 1], [E_CP, 1, 1, 1]]),
                
                "SECTIONS": #AREA, INERTIA 1, INERTIA 2, X GC,Y GC
                
                np.array([[A_V , I_V, I_V, 0, 0], [A_P , I_P, I_P, 0, 0]]),
                "PRESCRIBED DISPLACEMENTS": #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
                np.array([
                [0,0,0],
                [0,1,0],
                [0,2,0],
                [1,0,0],
                [1,1,0],
                [1,2,0]]),
                
                "ELEMENT LOADS": None,
                
                "NODAL LOADS":  #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
                np.array([
                [2,1,-F_I,0],
                [3,1,-F_II,0],
                [4,1,-F_II,0],
                [5,1,-F_II,0],
                [6,1,-F_II,0],
                [7,1,-F_II,0],
                [8,1,-F_I,0],
                [9,1,-F_I,0],
                [10,1,-F_II,0],
                [11,1,-F_II,0],
                [12,1,-F_II,0],
                [13,1,-F_II,0],
                [14,1,-F_II,0],
                [15,1,-F_I,0],
                [16,1,-F_I,0],
                [17,1,-F_II,0],
                [18,1,-F_II,0],
                [19,1,-F_II,0],
                [20,1,-F_II,0],
                [21,1,-F_II,0],
                [22,1,-F_I,0],
                [23,1,-F_I,0],
                [24,1,-F_II,0],
                [25,1,-F_II,0],
                [26,1,-F_II,0],
                [27,1,-F_II,0],
                [28,1,-F_II,0],
                [29,1,-F_I,0],
                [30,1,-F_I,0],
                [31,1,-F_II,0],
                [32,1,-F_II,0],
                [33,1,-F_II,0],
                [34,1,-F_II,0],
                [35,1,-F_II,0],
                [36,1,-F_I,0]]),
            
                "SPRINGS": None}    

    FRAME_00_RESULTS = MEF1D(DICTIONARY = FRAME_00)
    return FRAME_00_RESULTS

def ANALISE_MATRICIAL_ELS(DADOS):
    F_I = DADOS['CARGA_NODAL']
    F_II = 2 * DADOS['CARGA_NODAL']
    E_CV = DADOS['E_C'] * 1.00
    E_CP = DADOS['E_C'] * 1.00
    A_V = DADOS['A_V']
    A_P = DADOS['A_P']
    I_V = DADOS['I_V']
    I_P = DADOS['I_P']  
    FRAME_00 = {"TYPE_ELEMENT": 0,
                "TYPE_SOLUTION": 0,
                "N_NODES": 37,
                "N_MATERIALS": 2,
                "N_SECTIONS": 2,
                "N_ELEMENTS": 40,
                "N_DOFPRESCRIPTIONS": 6,
                "N_DOFLOADED": 35,
                "N_DOFSPRINGS": 0,
                "COORDINATES": 
                np.array([
                [0.00,0.00],
                [4.50,0.00],
                [0.00,3.00],
                [0.75,3.00],
                [1.50,3.00],
                [2.25,3.00],
                [3.00,3.00],
                [3.75,3.00],
                [4.50,3.00],
                [0.00,6.00],
                [0.75,6.00],
                [1.50,6.00],
                [2.25,6.00],
                [3.00,6.00],
                [3.75,6.00],
                [4.50,6.00],
                [0.00,9.00],
                [0.75,9.00],
                [1.50,9.00],
                [2.25,9.00],
                [3.00,9.00],
                [3.75,9.00],
                [4.50,9.00],
                [0.00,12.00],
                [0.75,12.00],
                [1.50,12.00],
                [2.25,12.00],
                [3.00,12.00],
                [3.75,12.00],
                [4.50,12.00],
                [0.00,15.00],
                [0.75,15.00],
                [1.50,15.00],
                [2.25,15.00],
                [3.00,15.00],
                [3.75,15.00],
                [4.50,15.00]]),

                "ELEMENTS": #NODE 1,NODE 2,MATERIAL ID,SECTION ID,HINGE ID NODE 1,HINGE ID NODE 2
                np.array([
                [0,2,1,1,0,0],#0
                [2,3,0,0,0,0],#1
                [3,4,0,0,0,0],#2
                [4,5,0,0,0,0],#3
                [5,6,0,0,0,0],#4
                [6,7,0,0,0,0],#5
                [7,8,0,0,0,0],#6
                [8,1,1,1,0,0],#7
                [2,9,1,1,0,0],#8
                [9,10,0,0,0,0],#9
                [10,11,0,0,0,0],#10
                [11,12,0,0,0,0],#11
                [12,13,0,0,0,0],#12
                [13,14,0,0,0,0],#13
                [14,15,0,0,0,0],#14
                [15,8,1,1,0,0],#15
                [9,16,1,1,0,0],#16
                [16,17,0,0,0,0],#17
                [17,18,0,0,0,0],#18
                [18,19,0,0,0,0],#19
                [19,20,0,0,0,0],#20
                [20,21,0,0,0,0],#21
                [21,22,0,0,0,0],#22
                [22,15,1,1,0,0],#23
                [16,23,1,1,0,0],#24
                [23,24,0,0,0,0],#25
                [24,25,0,0,0,0],#26
                [25,26,0,0,0,0],#27
                [26,27,0,0,0,0],#28
                [27,28,0,0,0,0],#29
                [28,29,0,0,0,0],#30
                [29,22,1,1,0,0],#31
                [23,30,1,1,0,0],#32
                [30,31,0,0,0,0],#33
                [31,32,0,0,0,0],#34
                [32,33,0,0,0,0],#35
                [33,34,0,0,0,0],#36
                [34,35,0,0,0,0],#37
                [35,36,0,0,0,0],#38
                [36,29,1,1,0,0]]),#39

                "MATERIALS": #YOUNG, POISSON,DENSITY, THERMAL COEFFICIENT
                np.array([[E_CV, 1, 1, 1], [E_CP, 1, 1, 1]]),
                
                "SECTIONS": #AREA, INERTIA 1, INERTIA 2, X GC,Y GC
                
                np.array([[A_V , I_V, I_V, 0, 0], [A_P , I_P, I_P, 0, 0]]),
                "PRESCRIBED DISPLACEMENTS": #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
                np.array([
                [0,0,0],
                [0,1,0],
                [0,2,0],
                [1,0,0],
                [1,1,0],
                [1,2,0]]),
                
                "ELEMENT LOADS": None,
                
                "NODAL LOADS":  #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
                np.array([
                [2,1,-F_I,0],
                [3,1,-F_II,0],
                [4,1,-F_II,0],
                [5,1,-F_II,0],
                [6,1,-F_II,0],
                [7,1,-F_II,0],
                [8,1,-F_I,0],
                [9,1,-F_I,0],
                [10,1,-F_II,0],
                [11,1,-F_II,0],
                [12,1,-F_II,0],
                [13,1,-F_II,0],
                [14,1,-F_II,0],
                [15,1,-F_I,0],
                [16,1,-F_I,0],
                [17,1,-F_II,0],
                [18,1,-F_II,0],
                [19,1,-F_II,0],
                [20,1,-F_II,0],
                [21,1,-F_II,0],
                [22,1,-F_I,0],
                [23,1,-F_I,0],
                [24,1,-F_II,0],
                [25,1,-F_II,0],
                [26,1,-F_II,0],
                [27,1,-F_II,0],
                [28,1,-F_II,0],
                [29,1,-F_I,0],
                [30,1,-F_I,0],
                [31,1,-F_II,0],
                [32,1,-F_II,0],
                [33,1,-F_II,0],
                [34,1,-F_II,0],
                [35,1,-F_II,0],
                [36,1,-F_I,0]]),
            
                "SPRINGS": None}    

    FRAME_00_RESULTS = MEF1D(DICTIONARY = FRAME_00)
    return FRAME_00_RESULTS

def ANALISE_MATRICIAL_VIGA_1(DADOS):
    F_I = DADOS['CARGA_NODAL']
    F_II = 2 * DADOS['CARGA_NODAL']
    E_CV = DADOS['E_C'] * 1.00
    E_CP = DADOS['E_C'] * 1.00
    A_V = DADOS['A_V']
    A_P = DADOS['A_P']
    I_V = DADOS['I_V']
    I_P = DADOS['I_P']  
    FRAME_00 = {"TYPE_ELEMENT": 0,
                "TYPE_SOLUTION": 0,
                "N_NODES": 11,
                "N_MATERIALS": 2,
                "N_SECTIONS": 2,
                "N_ELEMENTS": 10,
                "N_DOFPRESCRIPTIONS": 8,
                "N_DOFLOADED": 7,
                "N_DOFSPRINGS": 0,
                "COORDINATES": 
                np.array([
                [0.00,0.00],
                [4.50,0.00],
                [0.00,3.00],
                [0.75,3.00],
                [1.50,3.00],
                [2.25,3.00],
                [3.00,3.00],
                [3.75,3.00],
                [4.50,3.00],
                [0.00,6.00],
                [4.50,6.00]]),

                "ELEMENTS": #NODE 1,NODE 2,MATERIAL ID,SECTION ID,HINGE ID NODE 1,HINGE ID NODE 2
                np.array([
                [0,2,1,1,0,0],#0
                [2,3,0,0,0,0],#1
                [3,4,0,0,0,0],#2
                [4,5,0,0,0,0],#3
                [5,6,0,0,0,0],#4
                [6,7,0,0,0,0],#5
                [7,8,0,0,0,0],#6
                [8,1,1,1,0,0],#7
                [2,9,1,1,0,0],#8
                [8,10,1,1,0,0]]),#9

                "MATERIALS": #YOUNG, POISSON,DENSITY, THERMAL COEFFICIENT
                np.array([[E_CV, 1, 1, 1], [E_CP, 1, 1, 1]]),
                
                "SECTIONS": #AREA, INERTIA 1, INERTIA 2, X GC,Y GC
                
                np.array([[A_V , I_V, I_V, 0, 0], [A_P , I_P, I_P, 0, 0]]),
                
                "PRESCRIBED DISPLACEMENTS": #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
                np.array([
                [0,0,0],
                [0,1,0],
                [1,0,0],
                [1,1,0],
                [9,0,0],
                [9,1,0],
                [10,0,0],
                [10,1,0]]),
                
                "ELEMENT LOADS": None,
                
                "NODAL LOADS":  #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
                np.array([
                [2,1,-F_I,0],
                [3,1,-F_II,0],
                [4,1,-F_II,0],
                [5,1,-F_II,0],
                [6,1,-F_II,0],
                [7,1,-F_II,0],
                [8,1,-F_I,0]]),
            
                "SPRINGS": None}    

    FRAME_00_RESULTS = MEF1D(DICTIONARY = FRAME_00)
    return FRAME_00_RESULTS

In [6]:
G_MED = 1.06 * 4.60
Q_MED = 0.64
F_CKMED = 1.17 * 25 * 1E3
F_YKMED = 1.22 * 500 * 1E3
E_YKMED = 1.22 * 200 * 1E6
G = ['NORMAL', G_MED, (0.12 * G_MED)]        
Q = ['GUMBEL MAX', Q_MED, (0.40 * Q_MED)]
F_CK = ['NORMAL', F_CKMED, (F_CKMED * 0.15)]
F_YK = ['NORMAL', F_YKMED, (F_YKMED * 0.04)]
E_SK = ['NORMAL', E_YKMED, (E_YKMED * 0.04)]

#G = ['NORMAL', G_MED, 0]        
#Q = ['GUMBEL MAX', Q_MED, 0]
#F_CK = ['NORMAL', F_CKMED, 0]
#F_YK = ['NORMAL', F_YKMED, 0]
#E_SK = ['NORMAL', E_YKMED, 0]

POP = [100000]

SETUP = {'N_REP': len(POP),
        'POP': POP,
        'N_G': 4,
        'D': 5 ,
        'VARS': [G, Q, F_CK, F_YK, E_SK],
        'MODEL': 'MCS'}

In [7]:
def OBJ(X):
    # Dados fixos do sistema estrutural
    ALTURA_UTIL = 0.255
    LARGURA = 0.14
    GAMMA_C = 1.00
    GAMMA_S = 1.00
    GAMMA_F = 1.00
    PSI = 1.00
    VAO = 4.50   
    A_SV = 1.48 * 1E-4
    A_V = 0.14 * 0.30
    I_V = (0.14 * 0.30 ** 3) / 12
    A_P = 0.14 * 0.40
    I_P = (0.14 * 0.40 ** 3) / 12
    
    # Atribuição das variáveis aleatórias e variáveis de estado
    DEAD = X[0]
    LIVE = X[1]
    F_CK = X[2]
    F_YK = X[3]
    E_SK = X[4]
    E_CI, E_CS = MODULO_ELASTICIDADE_CONCRETO('GRA', F_CK, F_CK)
    ALPHA_MOD = E_SK / E_CS
    M_F = MOMENTO_FISSURACAO(2, F_CK, 0.30, 0.15, I_V)
    X_II, I_II = GEOMETRIC_PROPERTIES_STATE_II(0.30, 0.14, 0.14, 0, A_SV, 0, ALPHA_MOD, ALTURA_UTIL, 0)
    #print('dados',DEAD, LIVE, F_CK, F_YK, E_SK)
    #print('variáveis de estado', M_F, E_CI, E_CS, ALPHA_MOD, X_II, I_II)

    # Análise estrutural carga (G + Q) * GAMMA_F
    DADOS_0 = {'CARGA_NODAL': GAMMA_F * (DEAD + LIVE), 'E_C': E_CS, 'A_V': A_V, 'I_V': I_V, 'A_P': A_P, 'I_P': I_P}
    FRAME_0 = ANALISE_MATRICIAL_ELU(DADOS_0)
    
    # Resultados Viga V01 e P01 do 1º pavimento
    M_SD = max(FRAME_0[3]['M'])
    V_SD = max(FRAME_0[1]['V'])
    N_SD = np.abs(max(FRAME_0[0]['N']))
    
    # Análise estrutural carga G + Q * PSI  
    DADOS_1 = {'CARGA_NODAL': DEAD + LIVE * PSI, 'E_C': E_CI, 'A_V': A_V, 'I_V': I_V, 'A_P': A_P, 'I_P': I_P}
    FRAME_1 = ANALISE_MATRICIAL_ELS(DADOS_1)
    M_SDSER = max(FRAME_1[3]['M'])
    #print('momento serviço', M_SDSER)

    # Inércia e rigidez equivalente da peça
    if M_SDSER > M_F:
        I_M = BRANSON_INERTIA(M_F, M_SDSER, I_V, I_II)
        I_CNOVO = I_M
    else:
        I_CNOVO  = I_V
    #print('inercia', I_V, I_CNOVO, DEAD + LIVE * PSI)
    DADOS_2 = {'CARGA_NODAL': DEAD + LIVE * PSI, 'E_C': E_CI, 'A_V': A_V, 'I_V': I_CNOVO, 'A_P': A_P, 'I_P': I_P}
    FRAME_2 = ANALISE_MATRICIAL_VIGA_1(DADOS_2)
    
    DELTA_S = FRAME_2[3]['UY'][10]
    DELTA_SD = np.abs(DELTA_S) * 2.5
    #print('flecha', DELTA_S, DELTA_SD)
    # Capacidade do sistema ao momento fletor
    M_RD = MOMENTO_RESISTENE_MRD(LARGURA, ALTURA_UTIL, A_SV, F_YK, F_CK, GAMMA_S, GAMMA_C)
    
    # Capacidade do sistema ao esforço cortante
    V_RD2 = RESISTENCIA_BIELA_COMPRIMIDA(F_CK, LARGURA, ALTURA_UTIL, GAMMA_C)
    
    # Capacidade do sistema a deflexão
    DELTA_RD = FLECHA_LIMITE_DELTA_RD(VAO)
    
    # Capacidade do sistema a carga normal
    N_RD = VERIFICA_PILAR('CANTO', A_P, 2, E_SK, F_CK, GAMMA_C)
    
    # Montagem da capacidade e demanda do sistema em listas
    S = [M_SD, V_SD, DELTA_SD, N_SD]
    R = [M_RD, V_RD2, DELTA_RD, N_RD]
    G_0 = - M_RD + M_SD
    G_1 = - V_RD2 + V_SD
    G_2 = - DELTA_RD + DELTA_SD
    #print('flecha analise', DELTA_SD, DELTA_RD, G_2)
    G_3 = - N_RD + N_SD
    G = [G_0, G_1, G_2, G_3]
    return R, S, G

In [8]:
RESULTS_TEST = RASD_STOCHASTIC(SETUP, OBJ)

In [9]:
RESULTS_TEST[0]['TOTAL RESULTS']

,X_0,X_1,X_2,X_3,X_4,R_0,R_1,R_2,R_3,S_0,...,S_2,S_3,G_0,G_1,G_2,G_3,I_0,I_1,I_2,I_3
0,5.111408,0.869053,34084.698935,604497.216498,2.435449e+08,21.827047,283.749336,0.018,867.188864,13.757033,...,0.009842,162.5,-8.070014,-256.249336,-0.008158,-704.688864,0.0,0.0,0.0,0.0
1,3.473647,0.832853,33088.020840,604519.550094,2.423099e+08,21.798094,276.723664,0.018,847.105590,10.005115,...,0.004344,120.0,-11.792979,-256.723664,-0.013656,-727.105590,0.0,0.0,0.0,0.0
2,5.492395,0.653817,37159.268365,596034.624122,2.463005e+08,21.614469,304.939630,0.018,928.197759,15.007672,...,0.010848,180.0,-6.606797,-274.939630,-0.007152,-748.197759,0.0,0.0,0.0,0.0
3,4.460247,0.379204,25850.827194,614203.550585,2.580788e+08,21.836978,223.410488,0.018,723.438320,11.255754,...,0.008637,132.5,-10.581224,-200.910488,-0.009363,-590.938320,0.0,0.0,0.0,0.0
4,4.592333,0.516051,23339.319158,624761.669850,2.527350e+08,22.039334,203.965326,0.018,670.831206,12.506394,...,0.012432,150.0,-9.532940,-178.965326,-0.005568,-520.831206,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,4.651331,0.849149,24843.414799,600460.275567,2.587998e+08,21.325687,215.669095,0.018,704.903233,13.757033,...,0.014078,162.5,-7.568654,-188.169095,-0.003922,-542.403233,0.0,0.0,0.0,0.0
99996,5.621896,0.781387,24180.659689,655374.494610,2.676229e+08,23.099061,210.533520,0.018,700.189848,15.007672,...,0.016768,180.0,-8.091388,-180.533520,-0.001232,-520.189848,0.0,0.0,0.0,0.0
99997,4.829893,0.634272,25268.530225,593102.060894,2.429745e+08,21.102447,218.945410,0.018,698.817935,12.506394,...,0.011531,150.0,-8.596053,-193.945410,-0.006469,-548.817935,0.0,0.0,0.0,0.0
99998,6.086457,0.557449,34810.744115,614534.098040,2.362873e+08,22.194070,288.819068,0.018,874.509947,16.258312,...,0.014479,192.5,-5.935758,-256.319068,-0.003521,-682.009947,0.0,0.0,0.0,0.0


In [10]:
RESULTS_TEST[0]

{'TOTAL RESULTS':             X_0       X_1           X_2            X_3           X_4  \
 0      5.111408  0.869053  34084.698935  604497.216498  2.435449e+08   
 1      3.473647  0.832853  33088.020840  604519.550094  2.423099e+08   
 2      5.492395  0.653817  37159.268365  596034.624122  2.463005e+08   
 3      4.460247  0.379204  25850.827194  614203.550585  2.580788e+08   
 4      4.592333  0.516051  23339.319158  624761.669850  2.527350e+08   
 ...         ...       ...           ...            ...           ...   
 99995  4.651331  0.849149  24843.414799  600460.275567  2.587998e+08   
 99996  5.621896  0.781387  24180.659689  655374.494610  2.676229e+08   
 99997  4.829893  0.634272  25268.530225  593102.060894  2.429745e+08   
 99998  6.086457  0.557449  34810.744115  614534.098040  2.362873e+08   
 99999  4.568744  0.692379  30740.196713  657819.912630  2.382140e+08   
 
              R_0         R_1    R_2         R_3        S_0  ...       S_2  \
 0      21.827047  283.7493